# Getting Started with DeepOrigins
---
*How to go from simple **matrix multiplication** and basic **backpropagation** to ResNets, Cyclical Learning Rate Policies and breaking the **State-of-the-Art in Imagenet**, all from scratch!*

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Getting-Started-with-DeepOrigins" data-toc-modified-id="Getting-Started-with-DeepOrigins-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Getting Started with DeepOrigins</a></span></li><li><span><a href="#Python-Data-Model" data-toc-modified-id="Python-Data-Model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Python Data Model</a></span><ul class="toc-item"><li><span><a href="#__new__-&amp;__init__" data-toc-modified-id="__new__-&amp;__init__-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span><code>__new__</code> &amp;<code>__init__</code></a></span></li><li><span><a href="#__repr__-&amp;-__str__" data-toc-modified-id="__repr__-&amp;-__str__-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span><code>__repr__</code> &amp; <code>__str__</code></a></span></li><li><span><a href="#__len__-&amp;-__getitem__" data-toc-modified-id="__len__-&amp;-__getitem__-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span><code>__len__</code> &amp; <code>__getitem__</code></a></span></li><li><span><a href="#__call__" data-toc-modified-id="__call__-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span><code>__call__</code></a></span></li><li><span><a href="#__enter__,-__exit__-&amp;-__del__" data-toc-modified-id="__enter__,-__exit__-&amp;-__del__-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span><code>__enter__</code>, <code>__exit__</code> &amp; <code>__del__</code></a></span></li></ul></li><li><span><a href="#Python-Decorators" data-toc-modified-id="Python-Decorators-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Python Decorators</a></span></li><li><span><a href="#Callbacks-in-Python" data-toc-modified-id="Callbacks-in-Python-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Callbacks in Python</a></span><ul class="toc-item"><li><span><a href="#Events-&amp;-Callbacks" data-toc-modified-id="Events-&amp;-Callbacks-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Events &amp; Callbacks</a></span></li></ul></li><li><span><a href="#Statistical-Measures-of-Spread" data-toc-modified-id="Statistical-Measures-of-Spread-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Statistical Measures of Spread</a></span></li><li><span><a href="#Softmax" data-toc-modified-id="Softmax-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Softmax</a></span></li></ul></div>

# Python Data Model
---
**Python abstracts and represents all data as objects in compliance with the Von Neumann model: [Python Data Model](https://docs.python.org/3/reference/datamodel.html#object.__init__)**

Python classes can be extended with special behaviour using the `__dunder__` or data model methods such as `__init__`. We can overload operators in Python by using the specific `__dunder__` methods, each with a pre-defined name. For example, defining a `__add__` method in our class will add two objects of the class whenever a plus sign is encountered. 

Such *special* methods are also invoked by certain special syntax such as subscription (`arr[i]`) or during certain special context such as initialising a new object (`__init__`) or calling `len` on a container object such as a list.

We shall see some of the regularly used (by me) special methods below. For a complete and exhaustive guide please follow the documentation link given above. Let's begin!

## `__new__` &`__init__`

Now, we will play with a running example to demostrate these *magic* methods. We will design a `Polynomial` class to abstract the polynomials of math.

Many of you will be familiar with `__init__`. Whereas `__init__` is used to initialize the object, `__new__` is used to create the object itself, which can then be initialized. That's why `__init__` takes `self` as the first parameter but `__new__` takes `cls`. When `__new__` is called, the `self` instance doesn't exist yet. So the class is passed to `__new__` and then it creates a `self` instance to be used by `__init__`. 

Here, we will use them to construct only quadratic (2-degree) polynomials: 

In [42]:
class Polynomial:
    '''An abstraction of the polynomial'''
    
    def __new__(cls, *coefficients):
        '''An arbitary constraint for demonstration'''
        if len(coefficients) == 3:
            return super().__new__(cls)
        else:
            raise Exception("Polynomial must be quadratic with 3 non-zero coefficients")
        
    def __init__(self, *coefficients):
        self.coeff = [c for c in coefficients]

In [43]:
y = Polynomial(1,2,3)
y.coeff

[1, 2, 3]

In [44]:
y = Polynomial(2,5)

Exception: Polynomial must be quadratic with 3 non-zero coefficients

## `__repr__` & `__str__`

Let's make our `Polynomial` objects look like real polynomials.

Python gives us `__repr__` and `__str__` to represent the objects of our class to the outside world. So, what's the difference between the two:
* `__repr__`:  To design the "official" string representation which should, if possible, look like the valid Python expression used to create this object
* `__str__`: To design the "informal" string representation which prints pretty and looks closer to how we *think* about the object

In [45]:
class Polynomial:
    '''An abstraction of the polynomial'''
            
    def __init__(self, *coefficients):
        self.coeff = [c for c in coefficients]
    
    def __repr__(self):
        return f"Polynomial({self.coeff})"
    
    def __str__(self):
        return ' + '.join([f'{c}x^{i}'
                           for i,c in enumerate(self.coeff)])

In [46]:
y = Polynomial(1,2,3)
y

Polynomial([1, 2, 3])

In [47]:
print(y)

1x^0 + 2x^1 + 3x^2


## `__len__` & `__getitem__`

The `__len__` and `__getitem__` method along with some others (`__setitem__`, `__iter__`, `__contains__`) can be used to emulate container objects like tuples or lists i.e. sequences or even mappings like dictionaries.

Here, we will use these two *magic* methods to close the gap between our `Polynomial` class abstraction and the real math-y polynomials:
* Whenever someone calls the `len()` function on our `Polynomial` objects, we should return the nearest interpretation for the "length" of a polynomial. This must be the degree of the polynomial (highest power of x with a non-zero coefficient e.g. degree of $3x^{2} + 2x + 1$ is 2).
* Whenever someone tries to index into our polynomial object using `polynomial[i]` we simply return the coefficient  of that particular term e.g. 0th item of $3x^{2} + 2x + 1$ is 1, 2nd item of $3x^{2} + 2x + 1$ is 3. 

In [48]:
class Polynomial:
    '''An abstraction of the polynomial'''
            
    def __init__(self, *coefficients):
        self.coeff = [c for c in coefficients]
    
    def __len__(self):
        '''Degree of a polynomial'''
        return len(self.coeff)
    
    def __getitem__(self, i):
        return self.coeff[i]

In [49]:
y = Polynomial(1,2,3)

In [50]:
len(y)

3

In [51]:
y[2], y[1], y[0]

(3, 2, 1)

## `__call__`

Let's make our `Polynomial` objects do some real work. So what can we do with real polynomials like:

\begin{equation*}
y = 3x^{2} + 2x + 1
\end{equation*}

Well, we can compute the value of the polynomial at a given x like so:

\begin{align}
y(1) = 6 \\ 
y(2) = 17 \\
y(3) = 34 \\
\end{align}


Python has the `__call__` method to implement this behaviour. In general, defining a `__call__` method in a class makes it a *Callable* class. Then we can use the objects of this class just like we use functions e.g. f(x) -> obj(x). I find this really useful and really cool!  

In [119]:
class Polynomial:
    '''An abstraction of the polynomial'''
            
    def __init__(self, *coefficients):
        self.coeff = [c for c in coefficients]
    
    def __call__(self, x):
        return sum([c*(x**i) for i, c in enumerate(self.coeff)])

In [120]:
y = Polynomial(1,2,3)

In [121]:
y(1)

6

In [122]:
y(2)

17

In [123]:
y(3)

34

## `__enter__`, `__exit__` & `__del__`

Now, let's demonstrate the usage of `__enter__`, `__exit__` & `__del__` by implementing a random polynomial generator which works only within a context. 

When we enter the context using `with` we get either one of a linear, quadratic or cubic polynomial with random coefficients. We compute the value of the polynomial at x and optionally do some more operations in the body. When we exit the `with` context, `del` is automatically called for clean up. And next time we get a fresh new polynomial. 

In [161]:
import random

In [162]:
class RandomPolynomial():
    ''' Working within the context of a randomly defined polynomial'''
    
    def __enter__(self):
        degree = random.randint(1,3)
        coeffs = random.sample(range(1,9), k=degree)
        self.random_polynomial = Polynomial(*coeffs) 
        print(f"Entering with random coefficients: {coeffs}")
        return self.random_polynomial
    
    def __exit__(self, *args):
        '''__del__ is automatically called'''
        print("Exiting by cleaning up")
    
    def __del__(self):
        del(self.random_polynomial)
        print("Cleanup Complete")

In [168]:
with RandomPolynomial() as y:
    print(y(3))

Entering with random coefficients: [7, 5]
22
Exiting by cleaning up
Cleanup Complete


# Python Decorators
---

# Callbacks in Python
---

## Events & Callbacks

In [8]:
import ipywidgets as widgets

We create a graphical button using the ipywidgets.Button

In [9]:
b = widgets.Button(description="Click Here")
b

Button(description='Click Here', style=ButtonStyle())

However, it won't do anything no matter how many times you click on it. But something does happen when you click on it. The developers of the ipywidgets library have defined an event called `on_click()` which is called everytime the button registers a mouse click. If we, the users of this library, want something to happen when the button is clicked we can indeed open the source code of the library and manually edit the `on_click()` function. But that's gonna be a mess, a disaster waiting to happen. But the developers of ipywidgets created this library for users to create their own buttons with their own behaviours. Afterall we could actually create our own button, but how do we define it's behaviour?  

The ipywidgets' developers simply want us to write a function `f()` that implements our custom behaviour and pass the function `f()` to the `on_click()` function. The developers have promised us that `on_click()` will call our function `f()` back. That's a callback!

In [10]:
def f(o): print("Custom Click Click!")

In [11]:
b.on_click(f)
b

Button(description='Click Here', style=ButtonStyle())

Custom Click Click!


## 

# Statistical Measures of Spread
---

# Softmax
---